# Reproducible MRI Reconstruction with BART

[Jon Tamir](mailto:jtamir@utexas.edu), Chandra Family Department of Electrical and Computer Engineering, The University of Texas at Austin

Based off of previous tutorials from other contributors:  

### MRITogether 2021
[MRITogether 2021 Tutorial](https://github.com/mrirecon/bart-workshop/tree/master/mri_together_2021)

[Martin Uecker](mailto:uecker@tugraz.at)$^{\dagger,*,+}$, [Nick Scholand](mailto:scholand@tugraz.at)$^{*,+}$, [Moritz Blumenthal](mailto:moritz.blumenthal@med.uni-goettingen.de)$^*$, [Xiaoqing Wang](mailto:xiaoqing.wang@med.uni-goettingen.de)$^{*,+}$

$^{\dagger}$Graz University of Technology, $^*$University Medical Center Göttingen, $^+$German Centre for Cardiovascular Research, Partner Site Göttingen

### BART Webinar 1, 2020
[BART Webinar 1 Tutorial](https://github.com/mrirecon/bart-webinars/tree/master/webinar1)

[Max Litster](mailto:maxlitster@berkeley.edu), UC Berkeley

The purpose of this notebook is to introduce the [BART toolbox](https://mrirecon.github.io/bart), and its use as a tool for reproducible research in MRI.

We recommend checking other points of reference, primarily available in the following locations:
- [BART Workshops](https://github.com/mrirecon/bart-workshop)
- [BART Webinars](https://github.com/mrirecon/bart-webinars) (and [webinar recordings](https://www.youtube.com/@bart-toolboxdepartmental7435/playlists)\)


## Installation
__New:__ BART Version 0.9.00 has been released! Check the announcement on our [mailing list](https://lists.eecs.berkeley.edu/sympa/arc/mrirecon/2023-12/msg00000.html).

This version has been archived at CERN as:  

BART: version 0.9.00 (2023) DOI:10.5281/zenodo.10277939



### Local Usage
- Install bart from its [github repository](https://github.com/mrirecon/bart)
- Set the `TOOLBOX_PATH` to the BART directory and add it to the `PATH`

```bash
export TOOLBOX_PATH=/path/to/bart  
export PATH=$TOOLBOX_PATH:$PATH
```

Although the simplest way to call the BART CLI tools is from a terminal, there are also wrapper functions that allow the tools to be used from Matlab and Python. These are located under the `$TOOLBOX_PATH/matlab` and `$TOOLBOX_PATH/python` directories.

### Online Usage
We recommend running BART locally when possible. For demonstration purposes, MyBinder and Google Colaboratory can be used to access a Jupyter instance with BART with a browser. In the following we install and configure BART for both.

The following code will automatically detect which service you are using.

In [ ]:
# Check if notebook runs on colab
import sys, os

os.environ['COLAB'] = 'true' if ('google.colab' in sys.modules) else 'false'
os.environ['CUDA'] = '0'

#### BART Installation

Here we install BARTs dependencies, clone its repository from github, and compile it.

In [ ]:
%%bash

# MyBinder has BART already installed via the container
if $COLAB; then

  # Install BARTs dependencies
  apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

  # Clone Bart
  [ -d /content/bart ] && rm -r /content/bart
  git clone https://github.com/mrirecon/bart/ bart &> /dev/null
fi

In [ ]:
%%bash

if $COLAB; then

  cd bart

  # Configuration
  COMPILE_SPECS=" PARALLEL=1
                  OPENBLAS=1
                  BLAS_THREADSAFE=1"

  printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local

  # Compile BART
  make &> /dev/null && echo ok
fi

#### Setup Environment for BART

After downloading and compiling BART, the next step simplifies the handling of BARTs command line interface inside an ipyhton jupyter notebook. We add the BART directory to the PATH variable and include the python wrapper for reading *.cfl files:

In [ ]:
if os.environ['COLAB'] == 'true':
    os.environ['TOOLBOX_PATH'] = "./bart"
    os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
    sys.path.append(os.environ['TOOLBOX_PATH'] + "/python/")

Check BART setup:

In [ ]:
%%bash
echo "# BART version: "
bart version

### Setup Visualization Helper

For this tutorial we will visualize some images. Therefore, we need a helper function and some python libraries.


In [ ]:
# More python libraries
import cfl
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image

def bart_plot(data, title=None, vmin=None, vmax=None, cmap='gray', cbar_label='', mag=True, fsize=10):

    # Import data
    data = np.abs(data) if mag else data

    fig = plt.figure(figsize=(fsize,fsize))
    ax1 = fig.add_subplot(111)
    im = ax1.imshow(data, interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
    if title:
      plt.title(title)
    # Style settings
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_label(cbar_label)
    cbar.ax.tick_params()

    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    ax1.xaxis.set_ticks_position('none')
    ax1.yaxis.set_ticks_position('none')
    ax1.set_axis_off()


    plt.show()

### Download Supporting Materials
For this tutorial, we also need to download some data from the GitHub repository.

In [ ]:
%%bash

# Download the required supporting material if it is not already there
[ ! -f data.zip ] && wget -q https://github.com/mrirecon/bart-workshop/raw/master/mri_together_2023/data.zip
unzip -n data.zip

## BART Fundamentals
BART provides a number of tools for MRI image reconstruction and multi-dimensional array manipulation. We can get a list of them by running `bart` with no arguments.

Note: The ! character indicates that the following cell should be executed in bash vs. the native python kernel. It is not necessary when running BART commands from your terminal.



In [ ]:
# get a list of BART commands by running bart with no arguments:
! bart

### BART Command Structure

All BART commands are structured in the following way:

> **`bart`** + **`command`** + **`options`** + **`input / output`**

Each BART command consists of a number of optional arguments, followed by input and output files. To get some information about a command, use the -h flag. Optional arguments are indicated by brackets, and files are indicated by carrot symbols.

For example, the following command demonstrates the full suite of options for generating a numerical phantom in the image or k-space domain:

In [ ]:
! bart phantom -h

The `phantom` tool includes the option `-k` to create it in k-space, and `-x` to specify the size.

## Python Bindings

BART comes equipped with Python bindings that easily integrate with popular scientific computing libraries in Python. This tutorial introduces the structure of the BART python module.

If you're running in MyBinder, the paths should be updated and you should be all set to run the following demos.

If you are running locally, you will need to directly import the bart module. To do this, you can add BART's python directory to your `PYTHONPATH` environment variable. Add the following line to your .bashrc to do this:

```bash
export PYTHONPATH="${TOOLBOX_PATH}/python:$PYTHONPATH"
```

Where `TOOLBOX_PATH` is the location of your BART install.

This way, we can import the bart module as well as cfl, our utility file for dealing with BART's data format: a .hdr header file and a .cfl file of complex-float data.

If you're already running the demo in MyBinder or Colab, you may have to manually modify the path, which we do here:


In [ ]:
if os.environ['COLAB'] == 'true':
    import os
    bart_python_path = '{}/python'.format(os.environ['TOOLBOX_PATH'])
    sys.path.append(bart_python_path)

After having updated your `PYTHONPATH`, the following cells should execute error-free:

In [ ]:
from bart import bart
import cfl

### BART Python Command Structure
The `bart` module allows a user to execute any BART command-line tool via the following format:

```python
<outputs> = bart(<nargout>, <command>, <arguments>, ...)
```
where:
- `<outputs>` are the data file outputs of a BART command
- `<nargout>` is the number of outputs expected by the command
- `<command>` is the command string with flags
- `<arguments>` are the data files passed in for use in the function

Similar to the bash interface, we can also get the help string in a similar way:

In [ ]:
bart(0, 'phantom -h')

### Create phantoms

We will use the phantom tool to create various types of phantoms

In [ ]:
# Create a "BART logo" phantom in the image domain
bart_phantom = bart(1, 'phantom -x 256 -B').T

bart_plot(bart_phantom, title='BART logo')


# Create a "BART logo" phantom in k-space
bart_phantom_ksp = bart(1, 'phantom -x 256 -B -k').T

bart_plot(bart_phantom_ksp, title='BART logo (kspace)', vmax=.02)


## File formats and metadata

BART data files are (typically) stored in two files, a header (`.hdr`) file which contains metadata (dimensions, file type, etc.) and a complex float (`.cfl`) file which contains the raw data. The header file can be opened and read as a normal text file, while the raw data is a binary file. The raw data is a multi-dimensional array of single-precision complex floats, stored in row-major order with interleaved real and imaginary components.

A Note about data formats: Saving and loading files are not Python in the purest sense, in that data is not stored in RAM the entire time during execution. Under the hood, the Python bindings are writing .hdr/.cfl files to and from the disk. A RAM disk can be used if users are looking to improve performance in this aspect.

### Writing to and from `.cfl` files

The `cfl` Python library has some helpful tools to save or load BART data into a numpy-compatible format. In the first example, we will create a complex float array in numpy and then load it into BART from the command line. In the second example, we generate an array via the CLI and load it into Python via the cfl library.

In [ ]:
A = np.array([1 + 1j, 2 + 3j, 3 + 5j])
cfl.writecfl('A', A)
! ls A.*
! bart show A

Note that when calling BART functions from the command-line, you do not need to include the extension. This is because BART understands to look for both the `.cfl` and `.hdr` files.

In [ ]:
! bart ones 2 3 3 B
B = cfl.readcfl('B')
B

Because the header file is a text file, we can directly print it:

In [ ]:
! cat B.hdr

Although not discussed here, BART can write to other file formats that might not have a text header. Thus, it is better to use the show command:

In [ ]:
! bart show -m B

Our dataset is 16-dimensional, but only the first two dimensions are non-singleton.

By convention, the dimensions are `[X, Y, Z, C, M, T, F, ...]`,
where `(X, Y, Z)` are the spatial matrix dimensions,  
`C` and `M` are the coil dimensions and ESPIRiT maps dimensions, respectively,  
`T` and `F` are used for echo times and coefficient dimensions,   
followed by other higher order dimensions such as flow encoding, etc.

When using the Python interface, note that higher-order singleton dimensions will be squeezed and thus might not show up as 16-dimensional.

### Using Bitmasks to select dimensions
Let's reconstruct our BART k-space phantom using a inverse Fast Fourier Transform (IFFT). Let's first see the options of the FFT command

In [ ]:
bart(0, 'fft -h')

The FFT command takes some optional arguments. It then takes three required arguments: `bitmask`, `<input>`, and `<output`>.

In [ ]:
# check the shape of our BART phantom
print(bart_phantom_ksp.shape)

In [ ]:
# Perform IFFT reconstruction

bart_phantom_rec = bart(1, 'fft -i 3', bart_phantom_ksp)
bart_plot(bart_phantom_rec, 'IFFT of BART k-space phantom')

Thus, we performed an inverse (`-i`) Fast Fourier Transform on the image dimensions **`(0, 1)`** specified by the bitmask **`3`**.


BART loops over dimensions selected by *bitmasks*. This is a powerful approach for perfoming multi-dimensional operations, as most tools will work on arbitrarily chosen dimensions.  

In our case, we wanted to perform an iFFT along dimensions 0 and 1, and the corresponding bitmask is calculated as:  
<center>
$ \text{bitmask}=2^{~0} + 2^{~1} = 3$
</center> <br>
BART also provides a command-line tool to calculate the bitmasks for specific dimensions.

In [ ]:
# Calculate bitmask for active dimensions 0 and 1
! bart bitmask 0 1

Let's compare the reconstructed BART phantom to the original image-domain phantom.

First we will create a difference array by subtracting the two images.

Next we will concatenate the  arrays along the second dimension so that we can plot them as one row:


In [ ]:
err_rec = bart_phantom - bart_phantom_rec
phantom_compare = np.concatenate((bart_phantom, bart_phantom_rec, err_rec), 1)
print(phantom_compare.shape)
bart_plot(phantom_compare, title='Image-domain phantom vs. bart phantom')


**Question: why are the two images not identical?**

## BART Examples


### Multi-coil Shepp Logan phantom

A complete tutorial for creating and reconstructing phantom data can be found in the [1st BART Webinar Materials](https://github.com/mrirecon/bart-webinars/blob/master/webinar1/day1_advanced_recon.ipynb)

We start with a simple example that already demonstrates some useful functions in BART:

- Creation of synthetic k-space data using the `phantom` tool
- Creation of subsampling patterns using the `usamp` tool
- Calibration of sensitivities using ESPIRIT using the `ecalib` tool
- Parallel imaging reconstruction using the `pics` tool

On the first step, we will create synthetic multi-coil k-space data using the `phantom` tool. We use the '-k' option to specify that we want to simulate k-space data, the -x option to set the matrix size and -s to set the number of coils.

In [ ]:
# convenience function for unraveling a multi-coil array for plotting
def unravel(arr):
  return np.reshape(arr.transpose((0,3,1,2)), (arr.shape[0], np.prod(arr.shape[1:])))

In [ ]:
num_coils = 8
num_samples = 256

# create fully-sampled Cartesian multi-coil phantom in k-space
shepp = bart(1, 'phantom -k -x {} -s {}'.format(num_samples, num_coils))
print(shepp.shape)

bart_plot(unravel(shepp), title='multi-channel k-space phantom', vmax=100)

Next, we create a subsampling pattern. We can use BART to create sampling patterns suitable for 3D Cartesian imaging using the `poisson` and `upat` commands. Of course, we can use any tools we want to create them. Here, I will show off some of BART's other tools to create an equispaced 2D sampling pattern with a fully sampled center. This is of course not the most efficient way to create it -- it's just a nice way to showcase the tools.

In [ ]:
acs_size = 20
phase_accel = 2

# create a fully sampled ACS region
if acs_size > 0:
  acs_array = bart(1, 'ones 2 {} {}'.format(num_samples, acs_size))
else:
  acs_array = bart(1, 'zeros 2 {} {}'.format(num_samples, 2))
print(acs_array.shape)

acs_region = bart(1, 'resize -c 1 {}'.format(num_samples), acs_array)
print(acs_region.shape)


# create a zeros array of size phase_accel - 1
zeros_array = bart(1, 'zeros 2 {} {}'.format(num_samples, phase_accel - 1))
if phase_accel == 2:
  zeros_array = zeros_array[:,None]
print(zeros_array.shape)

# create a ones array of size 1 and concatenate with zeros_array
ones_array = bart(1, 'ones 2 {} 1'.format(num_samples, 1))
print(ones_array.shape)
zo_array = np.concatenate((zeros_array, ones_array[:,None]), 1)
print(zo_array.shape)

# replicate the zo_array to create a full-sized mask
zo2_array = bart(1, 'repmat 2 {}'.format(num_samples // phase_accel), zo_array)
print(zo2_array.shape)
zo3_array = bart(1, 'reshape 6 {} 1'.format(num_samples), zo2_array)
print(zo3_array.shape)

# combine with acs region
mask = (zo3_array + acs_region) > 0

bart_plot(mask)

In [ ]:
shepp_upat = shepp * mask[...,None,None]
bart_plot(unravel(shepp_upat), title='sub-sampled k-space', vmax=100)

Note that we used numpy broadcasting to automatically apply the mask to each coil.

After creating the under-sample data, we now first perform a simple reconstruction using an inverse Fourier transform (`fft`) and root-sum-of-squares coil combination (`rss`).

In [ ]:
# simple IFFT + RSS reconstruction

# perform inverse FFT. Use the unitary flag for unitary transform
shepp_fft = bart(1, 'fft -iu 3', shepp_upat)

# RSS-combine
shepp_rss = bart(1, 'rss 8', shepp_fft)

bart_plot(shepp_rss, title='RSS Recon')

The result is (of course) not very good. The image is corrupted with aliasing artifacts due to the equispaced subsampling.

Let's try a more advanced reconstruction technique. We will first determine the coil sensitivities. Here, we use the ESPIRiT algorithm to obtain the sensitivity maps for each coil. We will now have a closer look at the ESPIRiT `ecalib` tool.

In [ ]:
bart(0, 'ecalib -h')

The default number of maps is 2. For simplicity we will only use one set of coil sensitivities in this example by using the option `-m 1`.

In [ ]:
# compute sensitivity maps with ESPIRiT
sens_maps = bart(1, 'ecalib -m1', shepp_upat)

bart_plot(unravel(sens_maps), title='Sensitivity maps')

Given the sensitivities, we can use the `pics` command to perform iterative parallel imaging reconstruction using various regularization terms. It solves the folllowing minimization problem:

$\arg\min_x ||PFS x - y|| + \lambda R(x)$, where:
- $P$ is the sampling operator
- $F$ is the Fourier transform
- $S$ is the multiplication with the sensitivity maps
- $x$ is the unknown image
- $y$ are the acquired multi-coil k-space data
- $R$ is a regularization functional
- $\lambda$ is the regularization weighting parameter

A popular regularization is $\ell_2$, i.e. $R(x) = ||x||_2^2$.

The objective function then corresponds to a SENSE reconstruction.

In [ ]:
lamda = 0.005
num_iter = 50

# parallel imaging reconstruction using l2 regularization
shepp_l2 = bart(1, 'pics -S -l2 -r {} -i {}'.format(lamda, num_iter), shepp_upat, sens_maps)

bart_plot(shepp_l2, 'CG-SENSE reconstruction')

In [ ]:
# combine images for display

bart_plot(np.concatenate((shepp_rss, shepp_l2), 1), title='RSS vs CG-SENSE reconstructions')

## Reconstructing data from the scanner

Because the BART file format is simple, it is possible to read data from many different sources. The BART command includes the `twixread` tool for basic reading of Siemens `.dat` files. We also include GE Orchestra libraries for reading P-File and ScanArchives, called [ox-bart](https://github.com/mrirecon/ox-bart).

In the data folder, we have already converted a k-space dataset from a vendor-specific format to BART format. The files are the following:

- `ksp`: Raw k-space data
- `ref`: Autocalibration reference scan
- `noise`: Noise pre-scan

We will use all three for reconstruction

In [ ]:
! echo "ksp:"
! bart show -m ksp
! echo
! echo "ref:"
! bart show -m ref
! echo
! echo "noise:"
! bart show -m noise

In [ ]:
# load data into Python
ksp = cfl.readcfl('ksp')
print('ksp:', ksp.shape)

ref = cfl.readcfl('ref')
print('ref:', ref.shape)

noise = cfl.readcfl('noise')
print('noise:', noise.shape)

In [ ]:
bart_plot(unravel(ref), title='Ref', vmax=.001)
bart_plot(unravel(ksp), title='k-space', vmax=.0001)

Based on the array sizes and dimensions, we can infer that the readout dimension is 768 samples, the phase encode dimension is 382, and there are 16 coils. Notice that the reference scan has 33 phase-encode ACS lines.


Our scanner automatically doubles the FOV in the readout dimension.

As first steps, let's (1) crop the data in the readout dimension to FOV/2, and (2) zero-pad the ref data to be the same phase-encode size as the k-space data. Then let's perform IFFTs and look at them in the image domain.

In [ ]:
ksp_crop = bart(1, 'fft -u 1', bart(1, 'resize -c 0 {}'.format(ksp.shape[0]//2), bart(1, 'fft -iu 1', ksp)))
ref_crop = bart(1, 'fft -u 1', bart(1, 'resize -c 0 {}'.format(ref.shape[0]//2), bart(1, 'fft -iu 1', ref)))

ref_zpad = bart(1, 'resize -c 1 {}'.format(ksp_crop.shape[1]), ref_crop)
bart_plot(unravel(bart(1, 'fft -iu 3', ref_zpad)), title='Ref scan')
bart_plot(unravel(bart(1, 'fft -iu 3', ksp_crop)), title='coil images')

## Noise pre-whitening
Before reconstruction, it is a good idea to do noise pre-whitening. Most vendors should provide you with a noise pre-scan. If not, then you should complain to your vendor!

For more information about pre-whitening, as well as other excellent topics in parallel imaging, check out the [ISMRM 2015 Sunrise Course on The Image Reconstruction Pipeline](http://hansenms.github.io/sunrise/) by Michael Hansen.

We can use the BART `whiten` tool to inspect the covariance matrix and create a noise prewhitening matrix.

In [ ]:
bart(0, 'whiten -h')

In [ ]:
noise_cov = np.conj(noise.squeeze()).T @ noise.squeeze() / (noise.shape[0] - 1)
bart_plot(noise_cov, cmap='viridis', vmin=0, title='Noise covariance matrix (before whitening)')

noise_white = bart(1, 'whiten -n', noise, noise)
noise_white_cov = np.conj(noise_white.squeeze()).T @ noise_white.squeeze() / (noise_white.shape[0] - 1)
bart_plot(noise_white_cov, cmap='viridis', vmin=0, title='Noise covariance matrix (after whitening)')


In [ ]:
ksp_white = bart(1, 'whiten -n', ksp_crop, noise)
ref_white = bart(1, 'whiten -n', ref_crop, noise)

ref_white_zpad = bart(1, 'resize -c 1 {}'.format(ksp_crop.shape[1]), ref_white)
bart_plot(unravel(bart(1, 'fft -iu 3', ref_zpad)), title='Ref coils (before whitening)', vmax=.00001)
bart_plot(unravel(bart(1, 'fft -iu 3', ref_white_zpad)), title='Ref coils (after whitening)', vmax=2)

#### Coil Compression
To reduce the size of our dataset and therefore also decrease the computational complexity, we perform a coil compression with the `cc` command.

We will do coil compression in 2 steps. We will first use ``cc` on the reference data to create the coil compression matrix. We will then use ``ccapply` to apply it to both the reference data and to the k-space data. This way we use the same coil compression matrix for both arrays.

By passing `-A` we choose to use all possible data and to reduce the dataset to 8 virtual coils with `-p`.


In [ ]:
cc_mat = bart(1, 'cc -A', ref_white)

ref_white_cc = bart(1, 'ccapply -p 8', ref_white, cc_mat)
ref_white_cc_zpad = bart(1, 'resize -c 1 {}'.format(ksp_crop.shape[1]), ref_white_cc)

ksp_white_cc = bart(1, 'ccapply -p 8', ksp_white, cc_mat)

print(ksp_white_cc.shape)

#### RSS reconstructions
Let's do simple RSS reconstructions of the reference and subsampled data

In [ ]:
ref_rss = bart(1, 'rss 8', bart(1, 'fft -iu 3', ref_white_cc_zpad)).real
ksp_rss = bart(1, 'rss 8 ', bart(1, 'fft -iu 3', ksp_white_cc)).real

bart_plot(np.concatenate((ref_rss, ksp_rss), 1), title='RSS of ref data vs RSS of subsampled data')

#### Coil Sensitivity Estimation

The coil profile estimation is similar to the case with the phantom data, except it is  applied to the ref scan.

In [ ]:
sens_maps = bart(1, 'ecalib -m1', ref_white_cc_zpad)
bart_plot(unravel(sens_maps), title='Coil-compressed sensitivity maps')

#### Parallel imaging reconstruction
We again use `pics` with l2 regularization. This time we use the ref data to get an approximate scale factor and pass it directly to the `pics` command.

In [ ]:
# get scale factor from the ref data
scale_factor = np.percentile(ref_rss, 99)
print('reconstruction scale factor: {}'.format(scale_factor))

In [ ]:
lamda = .0015
debug_level = 4
reco = bart(1, 'pics -d {} -l2 -r {} -S -w {}'.format(debug_level, lamda, scale_factor), ksp_white_cc, sens_maps)

#### Visualize reconstruction

In [ ]:
bart_plot(reco, title='CG-SENSE Reconstruction')